## Modeling Pipeline :
* 1.use item similarity to overcome cold start issue
* 2.pipeline: train model, save model, import new user data, load model for recommendation, output recommendation, 

In [1]:
import boto3
import pickle
import h5py
import tables
import numpy as np
import pandas as pd
import sklearn
import caserec
import json

In [2]:
import turicreate as tc

In [3]:
raw_data=pd.read_csv('training_data_3column_750.csv')

In [4]:
raw_data['rating'].describe()

count    4.009685e+07
mean     2.523576e+00
std      3.545649e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      6.500000e+00
max      1.000000e+01
Name: rating, dtype: float64

In [4]:
#raw_data_user_count=raw_data.groupby('game_id')['user_id'].count().reset_index()

In [5]:
#training_data_2k=raw_data[raw_data.game_id.isin(raw_data_user_count[raw_data_user_count.user_id>2000].game_id)]

In [6]:
#training_data_2k.shape

In [7]:
#training_data_2k.game_id.nunique()

In [8]:
#training_data_2k.to_csv('training_data_3column_2k_new.csv')

In [5]:
#actions = tc.SFrame.read_csv('training_data_3column_2k_new.csv')
actions = tc.SFrame.read_csv('training_data_3column_750.csv')
items = tc.SFrame.read_csv('game_info_750.csv')
df_items=pd.read_csv('game_info_750.csv')

Finished parsing file /Users/shujdong/w210_game_recommendation_local/modeling/training_data_3column_750.csv

Parsing completed. Parsed 100 lines in 1.34143 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 2112641 lines. Lines per second: 1.2215e+06

Read 17152920 lines. Lines per second: 2.43995e+06

...

Finished parsing file /Users/shujdong/w210_game_recommendation_local/modeling/game_info_750.csv

Parsing completed. Parsed 100 lines in 0.071921 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,list,float,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/shujdong/w210_game_recommendation_local/modeling/game_info_750.csv

Parsing completed. Parsed 9595 lines in 0.053078 secs.

In [6]:
df_items=df_items.fillna(0)

In [7]:
df_items.isnull().sum().sum()

0

In [8]:
df_items.shape

(9595, 8)

In [9]:
actions=actions[['game_id','user_id','rating']]
actions

game_id,user_id,rating
1,Lil Red Head,7.0
1,hsjx945,8.0
1,RuffRyder,0.0
1,SkywalterDBZ,7.0
1,Olmestig,8.0
1,grassquit,9.0
1,erzengel,0.0
1,Feeble,0.0
1,thisisadamj,0.0
1,Tiberiusfox,9.0


In [10]:
actions['rating'].tail()

dtype: float
Rows: 10
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [11]:
items_final=items.dropna()
items_final=items_final[['game_id','category']]

In [12]:


model = tc.factorization_recommender.create(actions,'user_id','game_id',target='rating',item_data=items_final,side_data_factorization=True)



Preparing data set.

Data has 40096849 observations with 486469 users and 9595 items.

Data prepared in: 33.0836s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 5.09437

Final training RMSE: 2.25099

In [13]:
#save model

model.save('game_rec_model_750_bgguser_sideinfo')

In [14]:
#load model
rec_model=tc.load_model('game_rec_model_750_bgguser_sideinfo')

In [15]:
rec_model

Class                            : FactorizationRecommender

Schema
------
User ID                          : user_id
Item ID                          : game_id
Target                           : rating
Additional observation features  : 0
User side features               : []
Item side features               : ['game_id', 'category']

Statistics
----------
Number of observations           : 40096849
Number of users                  : 486469
Number of items                  : 9595

Training summary
----------------
Training time                    : 1108.6337

Model Parameters
----------------
Model class                      : FactorizationRecommender
num_factors                      : 8
binary_target                    : 0
side_data_factorization          : 1
solver                           : auto
nmf                              : 0
max_iterations                   : 50

Regularization Settings
-----------------------
regularization                   : 0.0
regularization_type      

In [17]:
#try with random user input data
#new_obs_data = tc.SFrame({ 'user_id' : ['survey', 'survey', 'survey', 'survey', 'survey'],
                               # 'game_id' : ['4098', '9216', '475', '148228', '68448']})

new_obs_data = ['Tiberiusfox']


#recommend_games = rec_model.recommend_from_interactions([4098,9216,475])
recommend_games = rec_model.recommend(new_obs_data,k=50)
recommend_games

user_id,game_id,score,rank
Tiberiusfox,170216,8.546273234561069,1
Tiberiusfox,150376,8.539159300997836,2
Tiberiusfox,110327,8.313701155856235,3
Tiberiusfox,54043,8.203032973483188,4
Tiberiusfox,161970,8.078403475955112,5
Tiberiusfox,183394,7.896399024203403,6
Tiberiusfox,521,7.819412949755771,7
Tiberiusfox,147020,7.7076945335417815,8
Tiberiusfox,209010,7.6776347190886565,9
Tiberiusfox,70919,7.663321736529452,10


In [56]:
new_obs_data1 =  ['hsjx945']
recommend_games = rec_model.recommend(new_obs_data1,k=50)
recommend_games

user_id,game_id,score,rank
hsjx945,245354,8.899896505549533,1
hsjx945,245352,8.86388409443198,2
hsjx945,216092,8.856959703639133,3
hsjx945,258412,8.738543632701022,4
hsjx945,216091,8.628232124522311,5
hsjx945,245353,8.627075079157931,6
hsjx945,230914,8.611693266108615,7
hsjx945,203420,8.45142972774802,8
hsjx945,250780,8.39700496502219,9
hsjx945,216094,8.378449085429294,10


In [341]:
##backend flask import new user data and return 
from flask import Flask
from flask import request
app = Flask(__name__)

import turicreate as tc

import json

rec_model=tc.load_model('game_rec_model')

@app.route("/recommend", methods=['POST'])

SyntaxError: unexpected EOF while parsing (<ipython-input-341-726a4f329022>, line 12)

In [13]:
#function to import json of user's answers, run model prediction, 
def recommend():
#import new user survey answer
#new_obs_data = json.loads("../data/bgg_survey_result.json")

    json_file_path = "../data/bgg_survey_result.json"

    with open(json_file_path, 'r') as j:
         new_obs_data = json.loads(j.read())

    new_obs_data_new=[new_obs_data["user_id"]]
    #print(new_obs_data_new)
    filter_condt=tc.SFrame({"age_min": [new_obs_data["age"]["min"]],
                        "age_max": [new_obs_data["age"]["max"]],
                        "num_players_min": [new_obs_data["num_players"]["min"]],
                        "num_players_max": [new_obs_data["num_players"]["max"]],
                        "play_time_min": [new_obs_data["play_time"]["min"]],
                        "play_time_max": [new_obs_data["play_time"]["max"]]})
    #print(filter_condt)
    recommend_items = rec_model.recommend(new_obs_data_new,k=50)
    
    #select 50 recommended game info 
    df_rec_game_info=df_items.loc[df_items['game_id'].isin(recommend_items['game_id'])]
    #print(df_rec_game_info)
    #filter out game based on user answers 
    df_items_filter=df_rec_game_info[(df_rec_game_info['age_min'] >= filter_condt['age_min'])&
                                (df_rec_game_info['min_ppl'] >= filter_condt['num_players_min']) &
                                (df_rec_game_info['max_ppl'] <= filter_condt['num_players_max']) &
                                (df_rec_game_info['avg_time']>= filter_condt['play_time_min'])&(df_rec_game_info['avg_time']<= filter_condt['play_time_max'])]
    #conver recommend_items to dataframe

    df_recommend_items=recommend_items.to_dataframe()
    #output top 5 games
    output=df_recommend_items.loc[df_recommend_items['game_id'].isin(df_items_filter.game_id)].sort_values('score',ascending=False).head(5)
    output_name=df_items_filter[df_items_filter['game_id'].isin(output['game_id'])][['game_id','game_title']]
    output=pd.merge(output,output_name, on='game_id')

    output['level'] = output['score'].where(~(output['score']>=9),"Extremely fits your taste ")
    output['level'] = output['level'].where(~((output['score']>=8)&(output['score']<9)),"Very much fits your taste")
    output['level'] = output['level'].where(~(output['score']<8),"Fits your taste")
    json_output=json.dumps({"game_id": list(output["game_id"]),"game_name": list(output["game_title"]),"level": list(output["level"])})
    #json_output=json.dumps({"game_id": list(output["game_id"]), "score": list(output["score"])})
    return json_output

In [14]:
#score=predicted rating
#https://apple.github.io/turicreate/docs/api/generated/turicreate.recommender.factorization_recommender.FactorizationRecommender.html#turicreate.recommender.factorization_recommender.FactorizationRecommender
recommend()

['hsjx945']
+---------+---------+-----------------+-----------------+---------------+---------------+
| age_max | age_min | num_players_max | num_players_min | play_time_max | play_time_min |
+---------+---------+-----------------+-----------------+---------------+---------------+
|    11   |    6    |        4        |        2        |       60      |       30      |
+---------+---------+-----------------+-----------------+---------------+---------------+
[1 rows x 6 columns]

      Unnamed: 0  game_id                                       game_title  \
1103        3044     3406                                  Lines of Action   
1336        4443     4937  Formula Dé Circuits 19 & 20: Suzuka & Melbourne   
6337       72671   167355                                          Nemesis   
7649       83348   199478                                     Flamme Rouge   
7801       84491   203417                  Exit: The Game – The Secret Lab   
7802       84492   203420             Exit: The 

'{"game_id": [252328, 255664, 244521, 199478, 244916], "game_name": ["Star Wars: X-Wing (Second Edition)", "The Binding of Isaac: Four Souls", "The Quacks of Quedlinburg", "Flamme Rouge", "Drop It"], "level": ["Very much fits your taste", "Very much fits your taste", "Very much fits your taste", "Very much fits your taste", "Fits your taste"]}'